In [ ]:
import argparse
import csv
import matplotlib.pyplot as plt
import glob
import os
import json
import seaborn as sns
import pandas as pd
import mpld3
from IPython import display

In [ ]:
leonhard_directory = "../logs/island_scaling_Nov_15_003228"

In [ ]:
from process_log import Tags, Log

In [ ]:
tags = Tags("tags.hpp")

In [ ]:
all_names = os.listdir(leonhard_directory)

# Validate JSON
json_file = list(filter(lambda x: ".json" in x, all_names))
if len(json_file) == 0:
    print("Could not find JSON file in directory {}".format(leonhard_directory))
    exit(1)
if len(json_file) > 1:
    print("Found multiple JSON files ({}) in the directory {}".format(json_file, leonhard_directory))
    exit(1)
json_file = json_file[0]
with open(os.path.join(leonhard_directory, json_file)) as file:
    json_file = json.load(file)
    repetitions = json_file["repetitions"]

all_names = list(filter(lambda x: os.path.isdir(os.path.join(leonhard_directory, x)), all_names))
unique_names = list(set(map(lambda x: "_".join(x.split("_")[:-1]), all_names)))
unique_names

In [ ]:
all_data = list()
for run_name in unique_names[1:]:
    for repetition in range(repetitions):
        folder_name = run_name + "_" + str(repetition)
        folder_contents = os.listdir(os.path.join(leonhard_directory, folder_name))
        folder_contents = list(filter(lambda x: ".bin" in x, folder_contents))
        logs = [Log(os.path.join(leonhard_directory, folder_name, path), tags) for path in folder_contents]
        rank_runtimes = list(map(lambda x: x.get_wall_clock_durations('logging')[0], logs))
        max_time = max(rank_runtimes)
        name = run_name.split("_")
        all_data.append((name[0], name[1], max_time, repetition))

In [ ]:
df = pd.DataFrame(all_data, columns=["n", "data", "time", "rep"])
df

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(ax=ax, y="time", x="n", data=df[df.data == "berlin52csv"])
ax.set_title("TSP Graph: berlin52")
fig.savefig("runtime_island_scaling_berlin52.svg")
ax

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(ax=ax, y="time", x="n", data=df[df.data == "a280csv"])
ax.set_title("TSP Graph: a280")
fig.savefig("runtime_island_scaling_a280.svg")
ax